In [0]:
spark.sql("USE CATALOG adb_cliente_savana_prd")


In [0]:
bronze_transacoes = spark.table("savana_elenir_bronze.transacoes")
bronze_clientes = spark.table("savana_elenir_bronze.clientes")


In [0]:
from pyspark.sql.functions import col, lower, trim, regexp_replace

# Tratamento de Transações
silver_transacoes = bronze_transacoes \
    .withColumn("id", regexp_replace(col("_id.oid"), "[-_:\\s'\\/().$!@]", "")) \
    .withColumn("oid", col("_id.oid")) \
    .withColumn("Meio_de_Pgmto", lower(trim(col("Meio_de_Pgmto")))) \
    .withColumn("Tipo", lower(trim(col("Tipo")))) \
    .drop("_id") \
    .dropna(subset=["ID_Cliente", "ID_Transacao", "Valor", "Meio_de_Pgmto", "Tipo", "id"]) \
    .dropDuplicates(["ID_Transacao", "ID_Cliente", "Valor", "Meio_de_Pgmto", "Tipo"])

# Padronização dos meios de pagamento
silver_transacoes = silver_transacoes.replace({
    "credito": "cartao de crédito",
    "crédito": "cartao de crédito",
    "debito": "cartao de débito",
    "débito": "cartao de débito",
    "pix": "pix"
}, subset=["Meio_de_Pgmto"])

# Salvar
silver_transacoes.write.mode("overwrite").saveAsTable("savana_elenir_silver.transacoes_tratadas")


In [0]:
from pyspark.sql.functions import col, trim, initcap, regexp_replace, when, length, to_date, lit

# Tratamento de Clientes
silver_clientes = bronze_clientes \
    .withColumn("Nome", initcap(trim(col("Nome")))) \
    .withColumn("Cidade", initcap(trim(col("Cidade")))) \
    .withColumn("CPF", regexp_replace(trim(col("CPF")), "[^0-9]", "")) \
    .withColumn("Agencia", regexp_replace(trim(col("Agencia")), "[^0-9A-Za-z]", "")) \
    .withColumn(
        "dt_transacao", 
        when(col("dt_transacao").isNull(), to_date(lit("1900-01-01")))
        .otherwise(col("dt_transacao"))
    ) \
    .withColumn(
        "CPF_valido", 
        when(length(col("CPF")) == 11, True).otherwise(False)
    ) \
    .fillna({
        "Nome": "Não informado",
        "Cidade": "Não informado",
        "CPF": "00000000000",
        "Agencia": "0000",
        "Transacao": -1
    }) \
    .dropDuplicates(["ID", "CPF"])

# Salvar
silver_clientes.write.mode("overwrite").saveAsTable("savana_elenir_silver.clientes_tratados")


In [0]:
# Carregar tabelas tratadas
silver_clientes = spark.table("savana_elenir_silver.clientes_tratados")
silver_transacoes = spark.table("savana_elenir_silver.transacoes_tratadas")

# Join para enriquecimento
transacoes_enriquecidas = (
    silver_transacoes.join(
        silver_clientes,
        silver_transacoes["ID_Cliente"] == silver_clientes["ID"],
        how="left"
    )
    .drop(silver_clientes["ID"])
)

# Salvar a tabela enriquecida
spark.sql("DROP TABLE IF EXISTS savana_elenir_silver.transacoes_enriquecidas")

transacoes_enriquecidas.write.mode("overwrite").saveAsTable("savana_elenir_silver.transacoes_enriquecidas")

# Visualizar
display(transacoes_enriquecidas)


In [0]:
# Selecionar catálogo
spark.sql("USE CATALOG adb_cliente_savana_prd")
meu_nome = "elenir"



In [0]:
from pyspark.sql.functions import col, lower, trim, regexp_replace

# Tratamento - TRANSACOES
silver_transacoes = bronze_transacoes \
    .withColumn("id", col("_id.oid")) \
    .withColumn("id", regexp_replace(col("id"), "[-_:\\s'\\/().$!@]", "")) \
    .withColumn("Meio_de_Pgmto", lower(trim(col("Meio_de_Pgmto")))) \
    .withColumn("Tipo", lower(trim(col("Tipo")))) \
    .withColumn("oid", col("_id.oid")) \
    .drop("_id") \
    .dropna(subset=["ID_Cliente", "ID_Transacao", "Valor", "Meio_de_Pgmto", "Tipo", "id"]) \
    .dropDuplicates(["ID_Transacao", "ID_Cliente", "Valor", "Meio_de_Pgmto", "Tipo"])

silver_transacoes = silver_transacoes.replace({
    "credito": "cartao de crédito",
    "crédito": "cartao de crédito",
    "debito": "cartao de débito",
    "débito": "cartao de débito",
    "pix": "pix"
}, subset=["Meio_de_Pgmto"])

silver_transacoes.write.mode("overwrite").saveAsTable(f"{meu_nome}silver.transacoes_tratadas")

In [0]:
# ✔️ Importando bibliotecas necessárias
from pyspark.sql.functions import col, trim, initcap, regexp_replace, when, length, to_date, lit


# ✔️ Tratamento dos dados
silver_clientes = bronze_clientes \
    .withColumn("Nome", initcap(trim(col("Nome")))) \
    .withColumn("Cidade", initcap(trim(col("Cidade")))) \
    .withColumn("CPF", regexp_replace(trim(col("CPF")), "[^0-9]", "")) \
    .withColumn("Agencia", regexp_replace(trim(col("Agencia")), "[^0-9A-Za-z]", "")) \
    .withColumn(
        "dt_transacao", 
        when(col("dt_transacao").isNull(), to_date(lit("1900-01-01")))
        .otherwise(col("dt_transacao"))
    ) \
    .withColumn(
        "CPF_valido", 
        when(length(col("CPF")) == 11, True).otherwise(False)
    ) \
    .fillna({
        "Nome": "Não informado",
        "Cidade": "Não informado",
        "CPF": "00000000000",
        "Agencia": "0000",
        "Transacao": -1
    }) \
    .dropDuplicates(["ID", "CPF"])

# ✔️ Salvando na camada Silver (sobrescrevendo)
silver_clientes.write.mode("overwrite").saveAsTable(f"{meu_nome}silver.clientes_tratados")

# ✔️ Visualizando dados para conferência
display(silver_clientes)


In [0]:
silver_clientes.printSchema()

In [0]:
# Enriquecimento

silver_clientes = spark.table(f"{meu_nome}silver.clientes_tratados")
silver_transacoes = spark.table(f"{meu_nome}silver.transacoes_tratadas")

transacoes_enriquecidas = (
    silver_transacoes.join(
        silver_clientes,
        silver_transacoes["ID_Cliente"] == silver_clientes["ID"],
        how="left"
    )
    .drop(silver_clientes["ID"])
)

# Dropar a tabela antes de salvar
spark.sql(f"DROP TABLE IF EXISTS {meu_nome}silver.transacoes_enriquecidas")

# Salvar sobrescrevendo 
transacoes_enriquecidas.write.mode("overwrite").saveAsTable(f"{meu_nome}silver.transacoes_enriquecidas")


display(transacoes_enriquecidas)